In [1]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
os.chdir('../Dataset/DatasetInfo')

In [30]:
# info_gse19301 = pd.read_csv('GSE19301_series_matrix.txt', sep='\t', header=None)

file_path = 'GSE41848.txt'

# Dati che vogliamo estrarre
sample_ids = []
sample_characteristics = []

# Apriamo il file e leggiamo riga per riga
with open(file_path, 'r') as file:
    for line in file:
        # Verifica se la linea contiene gli ID dei campioni
        if line.startswith('!Series_sample_id'):
            sample_ids = line.strip().split(' ')[1:]  # Prendi gli ID dei campioni
        # Verifica se la linea contiene le caratteristiche dei campioni
        elif line.startswith('!Sample_characteristics_ch1'):
            sample_characteristics = line.strip().split('\t')[1:]  # Prendi le caratteristiche

app = ['GSM1026056'] + sample_ids
# Creiamo un DataFrame con gli ID e
#  le caratteristiche
data = {
    'Sample ID': app,
    'Characteristics': sample_characteristics
}

df = pd.DataFrame(data)
print(df)

# info_gse41848 = pd.read_table('GSE41848_series_matrix.txt', sep='\t', header=None)
# info_gse59867 = pd.read_table('GSE59867_series_matrix.txt', sep='\t', header=None)
# info_gse61240 = pd.read_table('GSE61240_series_matrix.txt', sep='\t', header=None)
# info_gse88794 = pd.read_table('GSE88794_series_matrix.txt', sep='\t', header=None)
# info_gse146383 = pd.read_table('GSE146383_series_matrix.txt', sep='\t', header=None)
# info_gse136411_GPL6104 = pd.read_table('GSE136411-GPL6104_series_matrix.txt', sep='\t', header=None)
# info_gse136411_GPL10558 = pd.read_table('GSE136411-GPL10558_series_matrix.txt', sep='\t', header=None)

       Sample ID   Characteristics
0     GSM1026056  "gender: female"
1     GSM1026057  "gender: female"
2     GSM1026058  "gender: female"
3     GSM1026059  "gender: female"
4     GSM1026060  "gender: female"
..           ...               ...
207   GSM1026263  "gender: female"
208   GSM1026264  "gender: female"
209   GSM1026265  "gender: female"
210   GSM1026266  "gender: female"
211  GSM1026267"  "gender: female"

[212 rows x 2 columns]


In [2]:
print(os.getcwd())
file_path = "../Dataset/MappedNormalized"
os.chdir(file_path)

c:\Users\samue\OneDrive\Documenti\Studio\Università\Tesi\Progetto\ScriptPython


I Dataset sono stati normalizzati e vi sono state aggiunte le annotazioni tramite degli script in R, per il dataset gse136411, siccome campionato con due tecnologie diverse sono stati eliminati i geni che risultavano avere SYMBOL diversi per ID uguali nelle due tecnologie

In [3]:
def header(df):
    names = df.columns.values
    names[0] = "SampleID"
    df.columns = names

In [7]:
gse41848 = pd.read_csv('GSE41848_mapped.csv')
header(gse41848)

os.chdir("../SampleNotation")
sampleNotation41848 = pd.read_csv("GSE41848_sampleClassification.txt", sep='\t', encoding='utf-8')
os.chdir("../MappedNormalized")

def rename(entry):
    patient_splitted = entry.split(',')[0].split(' ')
    patientID = patient_splitted[2] if len(patient_splitted) == 3 else patient_splitted[1]
    visit_splitted = entry.split(',')[1].split(' ')
    isFollowUp = False if 'Baseline' in visit_splitted[1] else True
    if 'MS' in patient_splitted[0]:
        return patientID+'-FollowUp_'+visit_splitted[3] if isFollowUp else patientID
    else:
        return patientID+'-Control-'+'FollowUp_'+visit_splitted[3] if isFollowUp else patientID+'-'+'Control'

sampleNotation41848 = sampleNotation41848.T
columns = ['Title', 'SampleID']
sampleNotation41848.columns = columns
sampleNotation41848.drop(sampleNotation41848.index[0], inplace=True)
sampleNotation41848['Title'] = sampleNotation41848['Title'].apply(rename)
gse41848['SampleID'] = gse41848['SampleID'].str.split('_').str[0]

gse41848 = sampleNotation41848.merge(gse41848, on='SampleID', how='inner')
gse41848['SampleID'] = gse41848['SampleID']+'_'+gse41848['Title'] 
gse41848.drop(columns=['Title'], inplace=True)

print(gse41848.shape)
print(sum(gse41848['SampleID'].str.contains('-Control')))
print(gse41848['SampleID'].values)
print(gse41848.describe()) #OK NUMERICO
gse41848.head()
os.chdir("../Annotated")
# gse41848.to_csv("GSE41848_filtered.csv")
os.chdir("../MappedNormalized")


(212, 17518)
79
['GSM1026056_600009.0001' 'GSM1026057_600009.0001-FollowUp_1'
 'GSM1026058_41461.0001' 'GSM1026059_41462.0001' 'GSM1026060_600029.0001'
 'GSM1026061_41485.0001' 'GSM1026062_41497.0001'
 'GSM1026063_41497.0001-FollowUp_1' 'GSM1026064_41497.0001-FollowUp_2'
 'GSM1026065_41509.0001' 'GSM1026066_41509.0001-FollowUp_2'
 'GSM1026067_41509.0001-FollowUp_1' 'GSM1026068_600096.0001'
 'GSM1026069_600096.0001-FollowUp_1' 'GSM1026070_600096.0001-FollowUp_2'
 'GSM1026071_600104.0001' 'GSM1026072_41510.0001'
 'GSM1026073_600130.0001-FollowUp_1' 'GSM1026074_600134.0001'
 'GSM1026075_600134.0001-FollowUp_1' 'GSM1026076_600134.0001-FollowUp_2'
 'GSM1026077_600139.0001' 'GSM1026078_600139.0001-FollowUp_1'
 'GSM1026079_600139.0001-FollowUp_2' 'GSM1026080_600141.0001-FollowUp_1'
 'GSM1026081_41527.0001-FollowUp_2' 'GSM1026082_600153.0001'
 'GSM1026083_600153.0001-FollowUp_1' 'GSM1026084_600153.0001-FollowUp_2'
 'GSM1026085_41293.0001' 'GSM1026086_41293.0001-FollowUp_1'
 'GSM1026087_41293.0

Filtro il dataset recuperando solo i pazienti QUIET, sono stati eseguiti diversi campionamenti a distanza di tempo quindi li considero come FOLLOW UP

In [5]:
gse19301 = pd.read_csv("GSE19301_mapped.csv")
header(gse19301)

os.chdir("../SampleNotation")
sampleNotation19301 = pd.read_csv("GSE19301_sampleClassification.csv")
os.chdir("../MappedNormalized")

def extract_donor(stringa):
    match = re.search(r'Donor:\s*(\d+).*?Visit Type:\s*([A-Za-z]+)', stringa)
    if match:
        return match.group(1)+'-'+match.group(2)
    else:
        return None
    
sampleNotation19301['Title'] = sampleNotation19301['Title'].apply(extract_donor)
sampleNotation19301['Number'] = sampleNotation19301['Title'].str.extract(r'(\d+)')
sampleQUIET = sampleNotation19301[['Accession', 'Title', 'Number']][sampleNotation19301['Title'].str.contains('QUIET')]

def replace(entry):
    return entry.replace('QUIET', 'Control')

sampleQUIET['Title'] = sampleQUIET['Title'].apply(replace)

counter = {}
for index, row in sampleQUIET.iterrows():
    number = row['Number']
    title = row['Title']
    if number not in counter:
        counter[number] = 1
    else:
        counter[number] += 1
        sampleQUIET.loc[index, 'Title'] = f"{title}-FollowUp_{counter[number] - 1}"

gse19301['SampleID'] = gse19301['SampleID'].str.split('_').str[0]
gse19301 = gse19301[gse19301['SampleID'].isin(sampleQUIET['Accession'])]
gse19301 = sampleQUIET[['Accession', 'Title']].merge(gse19301, left_on='Accession', right_on='SampleID', how='right')

gse19301.drop(columns=['SampleID'], inplace=True)
gse19301.rename(columns={'Accession': 'SampleID'}, inplace=True)
gse19301['SampleID'] = gse19301['SampleID']+'_'+gse19301['Title']
gse19301.drop(columns=['Title'], inplace=True)

print(gse19301.shape)
print(sum(gse19301['SampleID'].str.contains('-Control')))
gse19301.head()
print(gse19301['SampleID'].values)
print(gse19301.describe()) #OK NUMERICO
os.chdir("../Annotated")
# gse19301.to_csv("GSE19301_filtered.csv")
os.chdir("../MappedNormalized")

(394, 21057)
394
['GSM479037_673-Control' 'GSM479038_818-Control' 'GSM479039_453-Control'
 'GSM479040_452-Control' 'GSM479041_722-Control' 'GSM479042_653-Control'
 'GSM479043_166-Control' 'GSM479044_413-Control' 'GSM479046_822-Control'
 'GSM479049_54-Control' 'GSM479053_701-Control' 'GSM479054_651-Control'
 'GSM479058_818-Control-FollowUp_1' 'GSM479059_825-Control'
 'GSM479060_257-Control' 'GSM479061_251-Control' 'GSM479062_59-Control'
 'GSM479065_310-Control' 'GSM479066_117-Control' 'GSM479068_774-Control'
 'GSM479069_819-Control' 'GSM479070_402-Control' 'GSM479071_815-Control'
 'GSM479074_409-Control' 'GSM479076_315-Control' 'GSM479077_401-Control'
 'GSM479078_841-Control' 'GSM479079_257-Control-FollowUp_1'
 'GSM479081_407-Control' 'GSM479083_308-Control' 'GSM479085_51-Control'
 'GSM479086_102-Control' 'GSM479088_66-Control' 'GSM479090_470-Control'
 'GSM479092_454-Control' 'GSM479094_677-Control'
 'GSM479097_774-Control-FollowUp_1' 'GSM479098_859-Control'
 'GSM479099_358-Control' 'GS

In [6]:
gse59867 = pd.read_csv("GSE59867_mapped.csv")
header(gse59867)

gse59867 = gse59867[gse59867['SampleID'].str.startswith("GSM16")].reset_index(drop=True)

def rename(entry):
    sampleID = entry.split('_')[0]
    match = re.findall(r'GSM(\d+)_', entry)
    return sampleID+'_'+match[0]+'-'+'Control'

gse59867['SampleID'] = gse59867['SampleID'].apply(rename)


print(gse59867.shape)
print(sum(gse59867['SampleID'].str.contains('-Control')))
gse59867.head()
print(gse59867['SampleID'].values)
print(gse59867.describe()) #OK NUMERICO
os.chdir("../Annotated")
# gse59867.to_csv("GSE59867_filtered.csv")
os.chdir("../MappedNormalized")

(46, 22026)
46
['GSM1620759_1620759-Control' 'GSM1620760_1620760-Control'
 'GSM1620761_1620761-Control' 'GSM1620762_1620762-Control'
 'GSM1620763_1620763-Control' 'GSM1620764_1620764-Control'
 'GSM1620765_1620765-Control' 'GSM1620766_1620766-Control'
 'GSM1620767_1620767-Control' 'GSM1620768_1620768-Control'
 'GSM1620769_1620769-Control' 'GSM1620770_1620770-Control'
 'GSM1620771_1620771-Control' 'GSM1620772_1620772-Control'
 'GSM1620773_1620773-Control' 'GSM1620774_1620774-Control'
 'GSM1620775_1620775-Control' 'GSM1620776_1620776-Control'
 'GSM1620777_1620777-Control' 'GSM1620778_1620778-Control'
 'GSM1620779_1620779-Control' 'GSM1620780_1620780-Control'
 'GSM1620781_1620781-Control' 'GSM1620782_1620782-Control'
 'GSM1620783_1620783-Control' 'GSM1620784_1620784-Control'
 'GSM1620785_1620785-Control' 'GSM1620786_1620786-Control'
 'GSM1620787_1620787-Control' 'GSM1620788_1620788-Control'
 'GSM1620789_1620789-Control' 'GSM1620790_1620790-Control'
 'GSM1620791_1620791-Control' 'GSM1620792

In [7]:
gse61240 = pd.read_csv("GSE61240_mapped.csv")
header(gse61240)

def rename(entry):
    entry_splitted = entry.split('_')
    match = re.findall(r'patient(\d+).', entry_splitted[1])
    return entry_splitted[0]+'_'+match[0]

gse61240['SampleID'] = gse61240['SampleID'].apply(rename)

print(gse61240.shape)
print(sum(gse61240['SampleID'].str.contains('-Control')))
gse61240.head()
print(gse61240['SampleID'].values)
print(gse61240.describe()) #OK NUMERICO
os.chdir("../Annotated")
# gse61240.to_csv("GSE61240_filtered.csv")
os.chdir("../MappedNormalized")

(550, 44663)
0
['GSM1500081_325' 'GSM1500082_373' 'GSM1500083_86' 'GSM1500084_5'
 'GSM1500085_137' 'GSM1500086_144' 'GSM1500087_9' 'GSM1500088_263'
 'GSM1500089_345' 'GSM1500090_516' 'GSM1500091_437' 'GSM1500092_183'
 'GSM1500093_102' 'GSM1500094_549' 'GSM1500095_468' 'GSM1500096_218'
 'GSM1500097_245' 'GSM1500098_158' 'GSM1500099_107' 'GSM1500100_328'
 'GSM1500101_248' 'GSM1500102_400' 'GSM1500103_105' 'GSM1500104_138'
 'GSM1500105_529' 'GSM1500106_450' 'GSM1500107_272' 'GSM1500108_205'
 'GSM1500109_154' 'GSM1500110_23' 'GSM1500111_243' 'GSM1500112_175'
 'GSM1500113_510' 'GSM1500114_271' 'GSM1500115_520' 'GSM1500116_232'
 'GSM1500117_56' 'GSM1500118_316' 'GSM1500119_58' 'GSM1500120_399'
 'GSM1500121_317' 'GSM1500122_29' 'GSM1500123_55' 'GSM1500124_283'
 'GSM1500125_185' 'GSM1500126_67' 'GSM1500127_452' 'GSM1500128_482'
 'GSM1500129_435' 'GSM1500130_337' 'GSM1500131_44' 'GSM1500132_202'
 'GSM1500133_319' 'GSM1500134_483' 'GSM1500135_65' 'GSM1500136_156'
 'GSM1500137_431' 'GSM1500138_53

In [4]:
gse136411 = pd.read_csv("GSE136411_mapped.csv")
header(gse136411)

os.chdir("../SampleNotation")
sampleNotation136411_1 = pd.read_csv("GSE_136411_6104_sampleClassification.txt", sep='\t', header=None, index_col=False)
os.chdir("../MappedNormalized")

sampleNotation136411_1 = sampleNotation136411_1.T
sampleNotation136411_1.drop(sampleNotation136411_1.index[0], inplace=True)
sampleNotation136411_1.columns = ['SampleID', 'Accession', 'Title']

os.chdir("../SampleNotation")
sampleNotation136411_2 = pd.read_csv("GSE_136411_10558_sampleClassification.txt", sep='\t', header=None, index_col=False)
os.chdir("../MappedNormalized")

sampleNotation136411_2 = sampleNotation136411_2.T
sampleNotation136411_2.drop(sampleNotation136411_2.index[0], inplace=True)
sampleNotation136411_2.columns = ['SampleID', 'Accession', 'Title']

sampleNotation136411 = pd.concat([sampleNotation136411_1, sampleNotation136411_2], axis=0)
sampleNotation136411.reset_index(drop=True, inplace=True)

delete = []
for i in range(287, 314):
    delete.append("Sample"+str(i))

gse136411 = gse136411[~gse136411['SampleID'].isin(delete)]

columns = gse136411.columns.values[1:]
gse136411[columns] = gse136411[columns].replace(',', '.', regex=True).astype(float)
gse136411 = sampleNotation136411.merge(gse136411, on='SampleID', how='inner')
gse136411['SampleID'] = gse136411['Accession']+'_'+gse136411['Title']
gse136411.drop(columns=['Title', 'Accession'], inplace=True)

def rename(entry):
    entry_splitted = entry.split('_')
    match = re.findall(r'rep(\d+)', entry_splitted[3])
    followUp = entry_splitted[3].split('.')
    isControl = True if 'HC' in entry_splitted[2] else False
    if len(followUp) == 1:
        return '4-'+entry_splitted[0]+'_'+match[0]+'-Control' if isControl else entry_splitted[0]+'_'+match[0]
    else:
        return '4-'+entry_splitted[0]+'_'+match[0]+'-Control'+'-FollowUp_'+followUp[1] if isControl else entry_splitted[0]+'_'+match[0]+'-FollowUp_'+followUp[1]

gse136411['SampleID'] = gse136411['SampleID'].apply(rename)

#Gli ID dei pazienti sono errati, chiamare questa funzione dopo tutte le altre per sistemarli
def rename2(dataset, sampleNotation):
    sampleID = dataset['SampleID']
    for i in range(len(sampleID)):
        if sampleID.iloc[i].startswith('4'):
            oldID = sampleID.iloc[i]
            sampleID.iloc[i] = sampleID.iloc[i].split('-')[1].split('_')[0]
            newID = ''
            if sampleID.iloc[i] in sampleNotation['Accession'].values:
                sampleNot = sampleNotation[sampleNotation['Accession'] == sampleID.iloc[i]]
                match = re.search(r'\d+', sampleNot['SampleID'].iloc[0].split('.')[0].split('_')[0])
                newID = sampleID.iloc[i]+'_'+match.group(0)

                if 'HC' in sampleNot['Title'].iloc[0]:
                    newID += '-Control'

                if sampleNot['Title'].iloc[0].find('.') != -1:
                    newID += '-FollowUp_'+sampleNot['Title'].iloc[0].split('.')[1]

                sampleID.iloc[i] = newID

rename2(gse136411, sampleNotation136411)

print(gse136411.shape)
print(sum(gse136411['SampleID'].str.contains('-Control')))
gse136411.head()
print(gse136411['SampleID'].values)
print(gse136411.describe()) #OK 
os.chdir("../Annotated")
gse136411.to_csv("GSE136411_filtered.csv")
os.chdir("../MappedNormalized")

C:\Users\samue\AppData\Local\Temp\ipykernel_19180\2131249368.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampleID.iloc[i] = sampleID.iloc[i].split('-')[1].split('_')[0]
C:\Users\samue\AppData\Local\Temp\ipykernel_19180\2131249368.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampleID.iloc[i] = newID
C:\Users\samue\AppData\Local\Temp\ipykernel_19180\2131249368.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampleID

(309, 9358)
67
['GSM4048323_1' 'GSM4048324_2' 'GSM4048325_3' 'GSM4048326_3-FollowUp_1'
 'GSM4048327_4' 'GSM4048328_5' 'GSM4048329_5-FollowUp_1' 'GSM4048330_6'
 'GSM4048331_6-FollowUp_1' 'GSM4048332_7' 'GSM4048333_8' 'GSM4048334_9'
 'GSM4048335_10' 'GSM4048336_11' 'GSM4048337_12' 'GSM4048338_13'
 'GSM4048339_14' 'GSM4048340_15' 'GSM4048341_16' 'GSM4048342_17'
 'GSM4048343_18' 'GSM4048344_19' 'GSM4048345_20' 'GSM4048346_21'
 'GSM4048347_22' 'GSM4048348_23' 'GSM4048349_24' 'GSM4048350_25'
 'GSM4048351_26' 'GSM4048352_27' 'GSM4048353_28' 'GSM4048354_29'
 'GSM4048355_30' 'GSM4048356_31' 'GSM4048357_32' 'GSM4048358_33'
 'GSM4048359_34' 'GSM4048360_35' 'GSM4048361_36' 'GSM4048362_37'
 'GSM4048363_38' 'GSM4048364_39' 'GSM4048365_40' 'GSM4048366_41'
 'GSM4048367_42' 'GSM4048374_49' 'GSM4048375_50' 'GSM4048376_51'
 'GSM4048377_52' 'GSM4048383_1' 'GSM4048386_2' 'GSM4048387_3'
 'GSM4048388_4' 'GSM4048389_5' 'GSM4048390_6' 'GSM4048391_7'
 'GSM4048392_8' 'GSM4048393_9' 'GSM4048394_10' 'GSM4048395_11

In [9]:
gse146383 = pd.read_csv("GSE146383_mapped.csv")
header(gse146383)

os.chdir("../SampleNotation")
sampleNotation146383 = pd.read_csv("GSE146383_sampleClassification.txt", sep='\t', encoding='utf-8')
os.chdir("../MappedNormalized")

sampleNotation146383 = sampleNotation146383.T
sampleNotation146383.columns = sampleNotation146383.iloc[0]
sampleNotation146383.drop(sampleNotation146383.index[0], inplace=True)
sampleNotation146383 = sampleNotation146383[sampleNotation146383['Sample_title'].str.contains('ad|AD', na=False)]

gse146383['SampleID'] = gse146383['SampleID'].str.split('_').str[0]
gse146383 = sampleNotation146383[['Sample_title', 'Sample_geo_accession']].merge(gse146383, right_on='SampleID', left_on='Sample_geo_accession', how='inner')
gse146383.drop(columns=['Sample_geo_accession'], inplace=True)
gse146383['SampleID'] = gse146383['SampleID']+"_"+gse146383['Sample_title']
gse146383.drop(columns=['Sample_title'], inplace=True)

def rename(entry):
    entry_splitted = entry.split('_')
    match = re.findall(r'\s(\w+)', entry_splitted[1])
    isControl = True if 'ADC' in entry else False
    return entry_splitted[0]+'_'+match[0]+'-Control' if isControl else entry_splitted[0]+'_'+match[0]

gse146383['SampleID'] = gse146383['SampleID'].apply(rename)

print(gse146383.shape)
print(sum(gse146383['SampleID'].str.contains('-Control')))
gse146383.head()
print(gse146383['SampleID'].values)
print(gse146383.describe()) #OK NUMERICO
os.chdir("../Annotated")
# gse146383.to_csv("GSE146383_filtered.csv")
os.chdir("../MappedNormalized")

(61, 21056)
46
['GSM4379744_7618ADMS212' 'GSM4379746_11283ADMS213'
 'GSM4379750_6495ADMS211' 'GSM4379751_8125ADMS212'
 'GSM4379752_13653ADMS210' 'GSM4379753_1662ADMS213'
 'GSM4379757_10677ADMS210' 'GSM4379758_4956ADMS212'
 'GSM4379759_7675ADMS211' 'GSM4379760_7701ADMS213'
 'GSM4379761_6069ADMS213' 'GSM4379762_7820ADMS212'
 'GSM4379768_12833ADMS211' 'GSM4379769_13546ADMS210'
 'GSM4379770_5219ADMS211' 'GSM4379781_6291PDC212-Control'
 'GSM4379783_5645C108-Control' 'GSM4379784_12912C190-Control'
 'GSM4379785_4491C73-Control' 'GSM4379786_1568-Control'
 'GSM4379787_5368C98-Control' 'GSM4379788_13409NMS203-Control'
 'GSM4379789_1891-Control' 'GSM4379790_3069C55-Control'
 'GSM4379791_1571C44-Control' 'GSM4379792_4489C73-Control'
 'GSM4379793_1566C46-Control' 'GSM4379794_2466C48-Control'
 'GSM4379795_3326C58-Control' 'GSM4379796_4200C69-Control'
 'GSM4379797_12899C189-Control' 'GSM4379798_13408NMS203-Control'
 'GSM4379799_2720C50-Control' 'GSM4379800_3070C56-Control'
 'GSM4379801_2120C46-Contro

In [10]:
gse88794 = pd.read_csv("GSE88794_mapped.csv")
header(gse88794)

os.chdir("../SampleNotation")
mapping = pd.read_csv("GSE88794_sampleClassification.csv")
os.chdir("../MappedNormalized")

mapping = mapping[['Accession', 'Title']][mapping['Title'].str.contains('Control')]

def extract_group(title):
    match = re.search(r"(NT\w+_W\d+D\d+T)", title)
    if match:
        return match.group(1)
    else:
        return None
    
def cleanDF(df):
    df['Title'] = df['Title'].str.rstrip("_Control")
    df['Group'] = df['Title'].apply(extract_group)
    df['Title'] = df['Title'].str.replace("NT", "")
    for groupID, group in df.groupby('Group'):
        groupIndexes = group.index.tolist()
        j = 0
        for i, rows in zip(groupIndexes, group.iterrows()):
            rows = rows[1]
            match = re.search(r"(T\d)", rows['Title'])
            if match.group(1) == "T0": 
                df.at[i, 'Title'] = "NT"+re.sub(r"T\d+", '-Control', rows['Title'])
            else: 
                df.at[i, 'Title'] = "NT"+re.sub(r"T\d+", '-Control-FollowUp_'+str(j), rows['Title'])
            j = j+1
    df.drop(columns=['Group'], inplace=True)
            
cleanDF(mapping)

gse88794['SampleID'] = gse88794['SampleID'].str.split('_').str[0]
gse88794 = mapping.merge(gse88794, left_on='Accession', right_on='SampleID', how='inner')
gse88794.drop(columns=['Accession'], inplace=True)
gse88794['SampleID'] = gse88794['SampleID'] +"_"+ gse88794['Title']
gse88794.drop(columns=['Title'], inplace=True)

print(gse88794.shape)
print(sum(gse88794['SampleID'].str.contains('-Control')))
gse88794.head()
print(gse88794['SampleID'].values)
print(gse88794.describe()) #OK NUMERICO
os.chdir("../Annotated")
# gse88794.to_csv("GSE88794_filtered.csv")
os.chdir("../MappedNormalized")

(541, 22026)
541
['GSM2346494_NT010_W5D2-Control' 'GSM2346497_NT034_W5D2-Control'
 'GSM2346499_NT056_W5D2-Control' 'GSM2346500_NT066_W5D2-Control'
 'GSM2346501_NT077_W5D2-Control'
 'GSM2346508_NT010_W5D2-Control-FollowUp_1'
 'GSM2346511_NT034_W5D2-Control-FollowUp_1'
 'GSM2346513_NT056_W5D2-Control-FollowUp_1'
 'GSM2346514_NT066_W5D2-Control-FollowUp_1'
 'GSM2346515_NT077_W5D2-Control-FollowUp_1'
 'GSM2346522_NT010_W5D2-Control-FollowUp_2'
 'GSM2346525_NT034_W5D2-Control-FollowUp_2'
 'GSM2346527_NT056_W5D2-Control-FollowUp_2'
 'GSM2346528_NT066_W5D2-Control-FollowUp_2'
 'GSM2346529_NT077_W5D2-Control-FollowUp_2'
 'GSM2346536_NT010_W5D2-Control-FollowUp_3'
 'GSM2346539_NT034_W5D2-Control-FollowUp_3'
 'GSM2346541_NT056_W5D2-Control-FollowUp_3'
 'GSM2346542_NT066_W5D2-Control-FollowUp_3'
 'GSM2346543_NT077_W5D2-Control-FollowUp_3'
 'GSM2346550_NT010_W18D2-Control' 'GSM2346553_NT034_W18D2-Control'
 'GSM2346555_NT056_W18D2-Control' 'GSM2346556_NT066_W18D2-Control'
 'GSM2346557_NT077_W18D2-C